In [1]:
import time
import pandas as pd
from census import Census
import altair as alt
import numpy as np

# Census API access
api_key = "639f2aedf7c17b164527591258cda00b25249b4b"
c = Census(key=api_key)

In [25]:
# Commute Mode: Table B08006
transp_variables = {
    'B08006_001E': 'total',
    'B08006_001M': 'total_moe',
    'B08006_002E': 'vehicle',
    'B08006_002M': 'vehicle_moe',
    'B08006_009E': 'bus',
    'B08006_009M': 'bus_moe',
    'B08006_010E': 'subway',
    'B08006_010M': 'subway_moe',
    'B08006_011E': 'rail',
    'B08006_011M': 'rail_moe',
    'B08006_012E': 'lightrail',
    'B08006_012M': 'lightrail_moe',
    'B08006_013E': 'ferry',
    'B08006_013M': 'ferry_moe',
    'B08006_014E': 'bike',
    'B08006_014M': 'bike_moe',
    'B08006_015E': 'walk',
    'B08006_015M': 'walk_moe',
    'B08006_016E': 'vehicle_other',
    'B08006_016M': 'vehicle_other_moe',
    'B08006_017E': 'wfh',
    'B08006_017M': 'wfh_moe'
}
transp_columns_out = [
    'total', 'total_moe',
    'pct_rail_all', 'pct_rail_all_moe', 
    'pct_other', 'pct_other_moe',
    'pct_vehicle', 'pct_vehicle_moe',
    'pct_bus', 'pct_bus_moe', 
    'pct_walk_or_bike', 'pct_walk_or_bike_moe'
]

In [47]:
def combine_modes(in_df):
    '''
    Outputs: simplified mode breakdown with MOEs
    '''
    df = in_df.copy()
    
    ### AGGREGATE ESTIMATES
    # Define a list of columns to combine into "other"
    other_cols = ['ferry', 'vehicle_other', 'wfh']
    df['other'] = df[other_cols].sum(axis='columns')
    # Use a list comprehension to append "_moe" to all strings in our list
    other_moes = [f'{col}_moe' for col in other_cols]
    df['other_moe'] = (df[other_moes]**2).sum(axis='columns')**0.5

    # Rail aggregate
    rail_cols = ['subway', 'rail', 'lightrail']
    df['rail_all'] = df[rail_cols].sum(axis='columns')
    rail_moes = [f'{col}_moe' for col in rail_cols]
    df['rail_all_moe'] = (df[rail_moes]**2).sum(axis='columns')**0.5

    # Active aggregate
    active_cols = ['walk', 'bike']
    df['walk_or_bike'] = df[active_cols].sum(axis='columns')
    active_moes = [f'{col}_moe' for col in active_cols]
    df['walk_or_bike_moe'] = (df[active_moes]**2).sum(axis='columns')**0.5

    for group in ['rail_all', 'other', 'vehicle', 'bus', 'walk_or_bike']:
        # Calculate the proportion for this group
        df[f'pct_{group}'] = df[group] / df['total']
    
        # Calculate the MOE for this proportion
        df[f'pct_{group}_moe'] = (df[f'{group}_moe']**2 - df[f'pct_{group}']**2 * df['total_moe']**2)**0.5 / df['total']

        #NaN-out any too-low absolute n
        df.loc[df['total'] < 25, f'pct_{group}'] = float('NaN')
        df.loc[df['total'] < 25, f'pct_{group}_moe'] = float('NaN')
        
        #NaN-out any too-low moe
        df[f'pct_{group}_moe_ratio'] = df[f'pct_{group}_moe']/df[f'pct_{group}']
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}'] = float('NaN')
        df.loc[df[f'pct_{group}_moe_ratio'] > .4, f'pct_{group}_moe'] = float('NaN')
    
    return df

In [46]:
def combine_tracts(in_df):
    '''
    Outputs:
    A modified version of in_df with census tracts combined
    '''
    
    # Start by making a copy of in_df, so we don't destroy the original data.
    # df is also a nice short name we can use throughout this function
    df_copy = in_df.copy()
    df = pd.DataFrame(columns=transp_variables.values())

    ### CLEAN UNUSUAL MOES
    df_copy = df_copy.replace(-555555555.0, 0)

    for c in transp_variables.values():
        if "moe" not in c:
            # sum the totals
            df.at[0, c] = df_copy[c].sum()
        else:
            # sum of squares the moes
            df.at[0, c] = np.sqrt((df_copy[c]**2).sum())

    df_out = combine_modes(df)
    
    return df_out

In [3]:
def get_transpo_precombo(year_in, place_num):
    # for getting mode for census tracts before they are combined
    df = pd.DataFrame(
        c.acs5.get(
            list(transp_variables.keys()),
            {'for': place_num, 'in': 'state:06 county:013'},
            year=year_in
        )
    )
    df = df.rename(columns=transp_variables)
    df = df.drop(columns=["state", "county", "tract"])
    
    return df

In [37]:
def get_city_df(year_in):
    # City
    df = pd.DataFrame(
        c.acs5.get(
            list(transp_variables.keys()),
            {'for': 'place:60620', 'in': 'state:06'},
            year=year_in
        )
    )
    df = df.rename(columns=transp_variables)
    df_out = combine_modes(df)

    return df_out

In [29]:
# for 5yr ACS 2014 and 2019
# Get ACS Table B08006 for select tracts around the BART station
# county:013
# city: 60620
c_tracts = 'tract:375000, 376000, 377000, 374000, 381000'

df_tracts_2014 = get_transpo_precombo(2014, c_tracts)
df_tracts_2019 = get_transpo_precombo(2019, c_tracts)

In [36]:
#Clean up and combine census tracts
df_comb_2014 = combine_tracts(df_tracts_2014)
df_tracts_out_2014 = df_comb_2014[transp_columns_out]
df_tracts_out_2014.insert(0, "year", 2014)
df_comb_2019 = combine_tracts(df_tracts_2019)
df_tracts_out_2019 = df_comb_2019[transp_columns_out]
df_tracts_out_2019.insert(0, "year", 2019)

In [40]:
# for 5yr ACS 2014 and 2019
# Get ACS Table B08006 in Richmond City
df_city_2014 = get_city_df(2014)
df_city_2019 = get_city_df(2019)

In [41]:
df_city_out_2014 = df_city_2014[transp_columns_out]
df_city_out_2014.insert(0, "year", 2014)
df_city_out_2019 = df_city_2019[transp_columns_out]
df_city_out_2019.insert(0, "year", 2019)

In [42]:
#export settings, for a combined csv
df_tracts_out = pd.concat([df_tracts_out_2014, df_tracts_out_2019])
df_city_out = pd.concat([df_city_out_2014, df_city_out_2019])

In [43]:
df_city_out

,year,total,total_moe,pct_rail_all,pct_rail_all_moe,pct_other,pct_other_moe,pct_vehicle,pct_vehicle_moe,pct_bus,pct_bus_moe,pct_walk_or_bike,pct_walk_or_bike_moe
0,2014,47029.0,1275.0,0.088392,0.011905,0.048991,0.007789,0.791086,0.017841,0.045185,0.006736,0.026345,0.005397
0,2019,52482.0,1209.0,0.097957,0.011294,0.060325,0.008493,0.777162,0.015651,0.035974,0.006713,0.028581,0.007106


In [44]:
df_tracts_out

,year,total,total_moe,pct_rail_all,pct_rail_all_moe,pct_other,pct_other_moe,pct_vehicle,pct_vehicle_moe,pct_bus,pct_bus_moe,pct_walk_or_bike,pct_walk_or_bike_moe
0,2014,11052.0,715.523585,0.117535,0.030913,NaN,NaN,0.782211,0.021776,0.042526,0.014075,0.036102,0.011559
0,2019,12961.0,679.254003,0.098912,0.021272,0.046061,0.013964,0.792917,0.021894,0.032559,0.01138,NaN,NaN
